# Simulation of Call Option Prices via Stochastic Differential Equations

This notebook simulates and prices four types of call options using the **Euler–Maruyama** method in conjunction with **Monte Carlo simulations**. The asset dynamics follow the **Geometric Brownian Motion (GBM)** model.

## Options Priced
- European Call Option
- Arithmetic Asian Call Option
- Geometric Asian Call Option
- Lookback Call Option (Floating Strike)

## Assumptions
- $S_0 = 100$
- $K = 100$
- $r = 0.05$
- $\sigma = 0.2$
- $T = 1$ year
- 252 time steps (daily)
- 10,000 Monte Carlo simulations


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set random seed
np.random.seed(42)

# Parameters
S0 = 100        # initial stock price
K = 100         # strike price
r = 0.05        # risk-free rate
sigma = 0.2     # volatility
T = 1.0         # time to maturity
M = 252         # number of time steps
dt = T / M      # time step size
N = 10000       # number of simulations


In [ ]:
def simulate_gbm_paths(S0, r, sigma, T, M, N):
    dt = T / M
    S = np.zeros((N, M + 1))
    S[:, 0] = S0
    for t in range(1, M + 1):
        Z = np.random.standard_normal(N)
        S[:, t] = S[:, t - 1] * np.exp((r - 0.5 * sigma**2) * dt + sigma * np.sqrt(dt) * Z)
    return S


In [ ]:
def european_call(S, K):
    return np.maximum(S[:, -1] - K, 0)

def arithmetic_asian_call(S, K):
    avg_price = np.mean(S[:, 1:], axis=1)
    return np.maximum(avg_price - K, 0)

def geometric_asian_call(S, K):
    log_S = np.log(S[:, 1:])
    avg_log = np.mean(log_S, axis=1)
    geom_mean = np.exp(avg_log)
    return np.maximum(geom_mean - K, 0)

def lookback_call(S):
    min_price = np.min(S[:, 1:], axis=1)
    return np.maximum(S[:, -1] - min_price, 0)


In [ ]:
def monte_carlo_price(payoffs, r, T):
    return np.exp(-r * T) * np.mean(payoffs)


In [ ]:
paths = simulate_gbm_paths(S0, r, sigma, T, M, N)

european_price = monte_carlo_price(european_call(paths, K), r, T)
arithmetic_asian_price = monte_carlo_price(arithmetic_asian_call(paths, K), r, T)
geometric_asian_price = monte_carlo_price(geometric_asian_call(paths, K), r, T)
lookback_price = monte_carlo_price(lookback_call(paths), r, T)

print("European Call Option Price:", european_price)
print("Arithmetic Asian Call Option Price:", arithmetic_asian_price)
print("Geometric Asian Call Option Price:", geometric_asian_price)
print("Lookback Call Option Price (Floating Strike):", lookback_price)
